## 定义迷宫

In [9]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
import openai

openai.api_type = "azure"
openai.api_base = "https://bigo-chatgpt-9.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
# key1: 19ea901e8e10475da1bb0537abf8e5a4
# key2: 548e5c0c2aff453e932948927a27bde6
openai.api_key = "19ea901e8e10475da1bb0537abf8e5a4"

# role : system|user|assistant
gpt_config = {'engine': 'gpt-35-turbo',
              'role': 'user',
              }

def get_response(prompt=None, content=None):
    message_list = [
            {"role": "system",
            "content": f"{content}"},
            {"role": "user", "content": f"{prompt}"}
            ]
    response = openai.ChatCompletion.create(
        engine=gpt_config['engine'],
        temperature=0.6,
        messages=message_list
    )
    return response['choices'][0]['message']['content']

In [10]:
### 迷宫的边界矩阵和占有矩阵是用来在Web界面成相的，所以用矩阵表示。而表示一些活动、事件的是一个字典，那些活动事件也同时都是字典
### 这些都是一个现有的大型矩阵，存储在这个位置：.\generative_agents\environment\frontend_server\static_dirs\assets\the_ville\matrix\maze
### world\sector\arena大多是用来搞前端的展示的，所以这里不进行定义，game_object才有交互
maze_dict = {'world':int(),
            'sector':int(),
            'arena':int(),
            'game_object':int(),}
maze = [[dict() for i in range(10)] for j in range(10)]

### 简单定义一些对应的场景
game_obj = {'No':0,   
            'bed':1,
            'playground':2,}

In [11]:
### 除了一些特定的地方，其他全部都设置成草地
for i in range(len(maze)):
    for j in range(len(maze[0])):
        maze[i][j]['world'] = 0
        maze[i][j]['sector'] = 0
        maze[i][j]['arena'] = 0

### 假设我们的主角处在（5，5）这个方格中，他的能见度是1，也就是说他只能看到周围距离为1的方格的内容
### 我们把和他相邻的几个地方的场景作为能交互的场景

### Events 是一个list，它包含了在当前场地上发生的一系列事件。这里只用一个事件简单代替
maze[4][5]['game_object'] = 2
maze[4][5]['events'] = ['Jim is playing basketball with Elk']

maze[4][4]['game_object'] = 2
maze[4][4]['events'] = ['Elk is playing basketball with Jim']

maze[5][6]['game_object'] = 1
maze[5][6]['events'] = ['No one is sleeping']


# agents行为流程

## Perceive

In [17]:
### 在perceive过程中，agents先获取附近的场景信息
### loc是agents当前所处位置，attn_width是能见度
def get_nearby_events(maze, loc, attn_width):
    row, column = loc[0], loc[1]
    perceive = []
    for i in range(max(row-attn_width,0), min(9,row+attn_width)+1):
        for j in range(max(column-attn_width,0), min(9,column+attn_width)+1):
            if 'events' in maze[i][j]:
                perceive.append(maze[i][j]['events'])
    return perceive

In [18]:
perceived = get_nearby_events(maze, (5,5), 1)
perceived

[['Elk is playing basketball with Jim'],
 ['Jim is playing basketball with Elk'],
 ['No one is sleeping']]

In [22]:
### 接收到当前附近发生的信息后，会搜索过去发生的一些事，这些事存在于associative memory里
### 这些事会以一个总结的形式展现出来
### event1: Bob and Jack are talking about delicious food
### event2: Juicy is studing to prepare her test exam
### event3: Alex played basketball on the ground
associative_memory = ["Bob: Hey Jack, I just had the most delicious meal last night. You wouldn't believe it!\nJack: Oh, really? Tell me all about it, Bob. I'm always up for some foodie inspiration.\nBob: Well, it was at this new Italian restaurant downtown. I started with a caprese salad – fresh tomatoes, mozzarella, basil, and the most divine balsamic glaze. It was a burst of flavors in my mouth!\nJack: That sounds amazing! Caprese salads are always a good choice. What came next?\nBob: For the main course, I couldn't resist ordering the homemade pasta with a truffle cream sauce. It was like a symphony of earthy and creamy flavors. The truffle aroma was just intoxicating.\nJack: You had me at truffle cream sauce, Bob. I'm a sucker for anything with truffles. Did you save room for dessert?\nBob: Of course! I went all out and had the tiramisu. It was like a sweet, coffee-infused dream. The ladyfingers were soaked just right, and the mascarpone cheese was so velvety. I practically licked the plate clean!\nJack: Haha, it sounds like you had an incredible dining experience, Bob. I'm adding that restaurant to my list of places to try. Speaking of Italian food, have you ever tried making homemade pizza?\nBob: Oh, absolutely! Making homemade pizza is one of my favorite weekend activities. I love experimenting with different toppings. What's your go-to pizza recipe?\nJack: Well, my signature pizza has a classic tomato sauce, fresh mozzarella, some prosciutto, and a handful of arugula on top after baking. It's simple but oh-so-delicious. And the dough – it's all about the perfect crust.\nBob: That sounds fantastic, Jack. We should have a pizza night one of these days and compare recipes.\nJack: That's a great idea, Bob! We can invite some friends over, have a little friendly competition, and, of course, savor some mouthwatering homemade pizza. It's a plan.\nBob: I'm in! I can't wait for that pizza night. It's always a blast to enjoy delicious meals and good company.",
                      "Juice was perched at her desk, surrounded by a fortress of textbooks and study materials. She had a crucial test exam coming up, and it was time to get down to some serious study. With a determined look on her face, she knew that focusing would be key to her success.\nBefore diving into her books, Juice decided to set the mood for her study session. She reached for her favorite pair of over-ear headphones and plugged them into her laptop. As she put them on, she could feel the world around her fading away.\nShe carefully selected a playlist of instrumental music that she knew would help her concentrate. The gentle, ambient tunes began to flow through her headphones, enveloping her in a soothing auditory cocoon. The music was carefully curated to enhance her focus, and it was precisely what she needed to keep distractions at bay.",
                      "Alex headed to the local playground with a basketball in hand. The sun was shining, and there was no one else on the court. With a grin, Alex started shooting hoops, enjoying the simple pleasure of playing basketball alone. The rhythmic sound of the ball bouncing and the occasional swish as it went through the net filled the air, creating a sense of peaceful satisfaction. It was a perfect, carefree afternoon of enjoying the game they loved."
]
events_prompt = ''
for i in range(len(associative_memory)):
    event = associative_memory[i]
    events_prompt = events_prompt + f'events{i}: {event}'
summarize_prompt = 'Please summarize these events:' + events_prompt + 'Use one sentence to summarize each event'
summarized_associative_memory = get_response(summarize_prompt)
# print(summarize_prompt)
print(summarized_associative_memory)

Event 0: Bob and Jack discuss a delicious Italian meal and their love for homemade pizza.
Event 1: Juice sets the mood for her study session with instrumental music.
Event 2: Alex enjoys a carefree afternoon of playing basketball alone.


In [23]:
summarized_events = [eve for eve in summarized_associative_memory.split('\n')]
summarized_events

['Event 0: Bob and Jack discuss a delicious Italian meal and their love for homemade pizza.',
 'Event 1: Juice sets the mood for her study session with instrumental music.',
 'Event 2: Alex enjoys a carefree afternoon of playing basketball alone.']

## Retrieve

In [24]:
### 设定一个previous memory的库，对于这些previous memory，我们以总结的形式展现
### 代码里的previous memory有很多信息，memory本身，对memory思考thought，还有一些描述description，这里为了简单只用summarized memory
### previous memory都存在向量数据库，用的是向量数据库的cos的匹配算法搜索的，这里不好演示，所以只是让gpt对目前发生的事，自己去找一个最相关的然后筛选

previous_evns = ["Tom decided to spend a relaxed afternoon hanging out in his quiet suburban street. He sat on his porch steps, sipping a cold drink and watching as the world went by. The gentle breeze rustled the leaves on the trees, and neighbors strolled past, exchanging friendly hellos and catching up on the latest news. It was a simple and peaceful way for Tom to unwind and enjoy the tranquil atmosphere of his neighborhood.",
                "Sarah, an artist, was in her cozy studio, surrounded by canvases and colorful paints. She was passionately working on a large, abstract masterpiece, her emotions flowing onto the canvas with each brushstroke.",
                "John, a young chef, was in a bustling restaurant kitchen. He was orchestrating the dinner service, ensuring that each dish leaving the kitchen was a work of culinary art. The aromatic scents and clatter of pots and pans filled the air.",
                "Maria, a dedicated nurse, was on the night shift at the hospital. She moved from room to room, checking on patients, administering medications, and providing comforting words to those in need. The soft glow of monitors illuminated her caring face.",
                "Michael, a teenager, was on the school football field. He was practicing tirelessly, running drills and perfecting his throwing technique in preparation for the big game on the weekend.",
                "Lisa, an archaeologist, was in a remote desert excavation site. She carefully brushed away sand and uncovered ancient artifacts, her heart racing with the thrill of discovery.",
                "Carlos, a taxi driver, was navigating the busy city streets. He cruised through traffic, picking up passengers and delivering them to their destinations, while listening to a mix of radio tunes.",
                "Emily, a bookstore owner, was rearranging the shelves in her quaint shop. She took great pleasure in curating the displays, bringing together books on various subjects, creating an inviting atmosphere for her customers.",
                "Ahmed, a dedicated teacher, was in his classroom, delivering an engaging lesson to a group of eager students. He used props and interactive activities to make learning fun and memorable.",
                "Sophia, a wildlife photographer, was hidden in the foliage of a dense forest. She aimed her camera at a family of deer, patiently waiting for the perfect shot that would capture the beauty of nature.",
                ]
events_prompt = ''
for i in range(len(previous_evns)):
    event = previous_evns[i]
    events_prompt = events_prompt + f'events{i}: {event}'
summarize_prompt = 'Please summarize these events:' + events_prompt + 'Use one sentence to summarize each event.'
summarized_prev_mem = get_response(summarize_prompt)
print(summarized_prev_mem)

1. Tom relaxed on his porch in a peaceful suburban neighborhood.
2. Sarah passionately painted an abstract masterpiece in her cozy studio.
3. John orchestrated a busy kitchen to create culinary art in a bustling restaurant.
4. Maria cared for patients on the night shift at a hospital.
5. Michael tirelessly practiced football on a school field.
6. Lisa uncovered ancient artifacts in a remote desert excavation site.
7. Carlos navigated busy city streets as a taxi driver.
8. Emily curated book displays in her quaint bookstore.
9. Ahmed engaged his students with interactive activities in his classroom, while Sophia captured the beauty of wildlife in a dense forest.


In [16]:
prev_mem = [mem for mem in summarized_prev_mem.split('\n')]
print(prev_mem)

['1. Tom relaxed on his porch in his peaceful neighborhood.', '2. Sarah passionately worked on an abstract painting in her cozy studio.', '3. John orchestrated a busy dinner service in a bustling restaurant kitchen.', '4. Maria provided care and comfort to patients on the night shift at the hospital.', '5. Michael practiced tirelessly on the football field in preparation for a big game.', '6. Lisa uncovered ancient artifacts at a remote desert excavation site.', '7. Carlos navigated the busy city streets as a taxi driver.', '8. Emily curated book displays in her quaint bookstore.', '9. Ahmed delivered engaging lessons to his eager students in his classroom.']


In [24]:
### 对于每一个当下接收到的信息，也就是perceived，在所有过往的事件中找一件与之最类似的
### 假设对于3条接收到的事件，与之相关联的时间分别为 3，5，7
### 返回结果

reterived_mem = [prev_mem[i] for i in [3, 5, 7]]
reterived_mem

['4. Maria provided care and comfort to patients on the night shift at the hospital.',
 '6. Lisa uncovered ancient artifacts at a remote desert excavation site.',
 '8. Emily curated book displays in her quaint bookstore.']

## Plan

In [25]:
### Plan 的输入是人物persona，迷宫整体maze，事件new_day，和被查找到的记忆reterived
### 如果是第一天或者是新的一天，所有plan将会被重新制定，否则则是执行之前已经规划好的计划
### 首先根据persona的记忆，我们知道会有recency，importance等去评判国王记忆的优先级，这时候会对过往记忆打一个优先级的标签
### 因为是简陋版所以这里直接假定给出优先级，选最高的前两个

priority = {0:9, 1:8, 2:7}
focused_event = [reterived_mem[i] for i in [0,1]]
focused_event 

['4. Maria provided care and comfort to patients on the night shift at the hospital.',
 '6. Lisa uncovered ancient artifacts at a remote desert excavation site.']

In [ ]:
### 接下来设定react model， react mode是根据当前接收到的信息和迷宫信息判断的
reaction_mode = _should_react(persona, focused_event, personas)
### 比如如果检测到附近有人在旁边，react mode = chat，如果有床之类的可以交互的道具， react mode = play
### 如果是没有什么其他的，react mode = wait